In [1]:
import os
# go one directory up
os.chdir("..")

In [2]:
import polars as pl
import numpy as np
import datetime
from smap import *
import smap
import smap.helpers as helpers

In [3]:
# Function to generate timestamps every 15 minutes over a specified period
def generate_timestamps(start_date, weeks_duration):
    end_date = start_date + datetime.timedelta(weeks=weeks_duration)
    return pl.datetime_range(start=start_date, end=end_date, interval="15m", eager=True)

# Function to generate test data
def generate_test_data():
    # Generate data for 2 weeks in summer and 2 weeks in winter
    summer_start = datetime.datetime(2023, 6, 1)
    winter_start = datetime.datetime(2023, 12, 1)
    # Generate timestamps
    summer_dates = generate_timestamps(summer_start, 2)
    winter_dates = generate_timestamps(winter_start, 2)
    # Concatenate summer and winter dates
    dates = pl.concat([summer_dates, winter_dates])
    # Simulate consumption data
    np.random.seed(0)  # for reproducibility
    cons = np.random.rand(len(dates)) * 2.3   # Random values between 0 and 100
    temp = np.random.rand(len(dates)) * 10   # Random values between 0 and 100
    # Create DataFrame
    df = pl.DataFrame({"dt": dates, "cons": cons, "temp": temp})
    return df

# Test function template
def test_function(func, data):
    try:
        result = func(data)
        print(f"Test passed for {func.__name__}")
        return result
    except Exception as e:
        print(f"Test failed for {func.__name__}: {e}")

In [4]:
# Generate test data
test_data = generate_test_data()
# Create a sequence of row numbers
row_numbers = pl.arange(0, test_data.height, eager=True)

# Use `with_column` to modify the `cons` column conditionally
test_data = test_data.with_columns(
    pl.when(row_numbers < 20)  # Condition for the first 20 rows
    .then(pl.lit(None))        # Replace with None (null in Polars)
    .otherwise(pl.col("cons")) # Keep the original value for other rows
    .alias("cons")             # Apply the changes to the `cons` column
)
test_data

dt,cons,temp
datetime[μs],f64,f64
2023-06-01 00:00:00,null,4.343823
2023-06-01 00:15:00,null,5.753966
2023-06-01 00:30:00,null,4.671086
2023-06-01 00:45:00,null,7.530424
2023-06-01 01:00:00,null,0.059388
2023-06-01 01:15:00,null,4.346325
2023-06-01 01:30:00,null,3.375814
2023-06-01 01:45:00,null,6.091641
2023-06-01 02:00:00,null,7.759584


In [5]:
helpers.calc_features_consumption(test_data)

year,week,average_cons,max_cons,min_cons,average_cons_morning,average_cons_noon,average_cons_afternoon,average_cons_evening,average_cons_night,average_cons_wd,average_cons_weekend,cons_variance,mean_cor,t_wide_peaks,t_width_peaks,first_exceeding_1kW_hour,mean_residual_stl,mean_autocorrelation
i32,u32,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,f64,f64,f64,f64
2023,48,1.218186,2.296611,0.037381,0.997055,1.352355,1.088006,1.168859,1.357687,null,1.206937,0.457443,NaN,134,3.435897,null,0.235482,0.010225
2023,22,1.13397,2.297348,0.0108,1.131937,1.057382,1.115154,1.145092,1.17495,1.035243,1.185074,0.437807,NaN,null,null,5.0,null,null
2023,24,1.201697,2.295386,0.001528,1.298298,1.184336,1.348476,1.001102,1.18316,1.201697,null,0.499721,NaN,126,3.073171,0.0,0.224341,-0.010218
2023,50,1.189833,2.293207,0.002078,1.234593,1.199318,1.113629,1.237586,1.17366,1.190817,0.811984,0.430337,NaN,160,3.018868,0.0,0.270802,-0.000739
2023,49,1.152842,2.295858,0.00017,1.110777,1.157423,1.22149,1.210373,1.145038,1.123026,1.188261,0.446113,NaN,260,3.023256,0.0,0.29819,-0.018626
2023,23,1.133106,2.29956,0.001256,0.998984,1.089252,1.210546,1.158992,1.128175,1.127214,1.147144,0.454988,NaN,225,2.922078,0.25,0.272075,-0.012676


In [6]:
smap_functions_list = [ "c_week", "s_max", "s_min", "c_morning", "c_noon",
    "c_afternoon", "c_evening", "c_night", "c_weekday", "c_var_weekday",
    "s_wd_min", "s_wd_max", "c_wd_morning", "c_wd_noon", "c_wd_afternoon",
    "c_wd_evening", "c_wd_night", "c_weekend", "c_var_weekend", "s_we_min",
    "s_we_max", "c_we_morning", "c_we_noon", "c_we_afternoon", "c_we_evening",
    "c_we_night", "c_week_no_min", "s_max_no_min", "c_evening_no_min",
    "c_morning_no_min", "c_noon_no_min", "c_night_no_min", "r_mean_max",
    "r_min_mean", "r_night_mean", "r_morning_noon", "r_evening_noon",
    "r_mean_max_no_min", "r_evening_noon_no_min", "r_morning_noon_no_min",
    "r_day_night_no_min", "r_var_wd_we", "r_min_wd_we", "r_max_wd_we",
    "r_evening_wd_we", "r_night_wd_we", "r_noon_wd_we", "r_morning_wd_we",
    "r_afternoon_wd_we", "r_we_night_day", "r_we_morning_noon",
    "r_we_evening_noon", "r_wd_night_day", "r_wd_morning_noon",
    "r_wd_evening_noon", "s_sm_variety", "s_bg_variety", "s_day_diff",
    "s_variance", "s_var_wd", "s_var_we", "s_diff", "s_cor", "s_num_peaks",
    "s_q1", "s_q2", "s_q3", "c_max_avg", "c_min_avg", "s_number_zeros",
    "s_cor_wd", "s_cor_we", "s_cor_wd_we", "s_number_small_peaks",
    "s_number_big_peaks", "w_temp_cor_overall", "w_temp_cor_daily",
    "w_temp_cor_night", "w_temp_cor_daytime", "w_temp_cor_evening",
    "w_temp_cor_minima", "w_temp_cor_maxima", "w_temp_cor_maxmin",
    "w_temp_cor_weekday_weekend", "t_above_1kw", "t_above_2kw",
    "t_above_mean", "t_daily_max", "t_daily_min", "ts_stl_varRem",
    "ts_acf_mean3h", "ts_acf_mean3h_weekday", "t_wide_peaks",
    "t_width_peaks"]

# Testing each function
for func_name in smap_functions_list:  # Replace with your list of function names
    func = getattr(smap, func_name, None)
    if callable(func):
        test_function(func, test_data)
    else:
        print(f"Function {func_name} not found in module.")

Test passed for c_week
Test passed for s_max
Test passed for s_min
Test passed for c_morning
Test passed for c_noon
Test passed for c_afternoon
Test passed for c_evening
Test passed for c_night
Test passed for c_weekday
Test passed for c_var_weekday
Test passed for s_wd_min
Test passed for s_wd_max
Test passed for c_wd_morning
Test passed for c_wd_noon
Test passed for c_wd_afternoon
Test passed for c_wd_evening
Test passed for c_wd_night
Test passed for c_weekend
Test passed for c_var_weekend
Test passed for s_we_min
Test passed for s_we_max
Test passed for c_we_morning
Test passed for c_we_noon
Test passed for c_we_afternoon
Test passed for c_we_evening
Test passed for c_we_night
Test passed for c_week_no_min
Test passed for s_max_no_min
Test passed for c_evening_no_min
Test passed for c_morning_no_min
Test passed for c_noon_no_min
Test passed for c_night_no_min
Test passed for r_mean_max
Test passed for r_min_mean
Test passed for r_night_mean
Test passed for r_morning_noon
Test passe

In [7]:
merged_df = None

# Testing each function and merging the results
for func_name in smap_functions_list:
    func = getattr(smap, func_name, None)
    if callable(func):
        result_df = func(test_data)
        if merged_df is None:
            merged_df = result_df
        else:
            print(f"Output added for {func.__name__}")
            merged_df = merged_df.join(result_df, on=['year', 'week'], how='outer')
    else:
        print(f"Function {func_name} not found in module.")

merged_df

Output added for s_max
Output added for s_min
Output added for c_morning
Output added for c_noon
Output added for c_afternoon
Output added for c_evening
Output added for c_night
Output added for c_weekday
Output added for c_var_weekday
Output added for s_wd_min
Output added for s_wd_max
Output added for c_wd_morning
Output added for c_wd_noon
Output added for c_wd_afternoon
Output added for c_wd_evening
Output added for c_wd_night
Output added for c_weekend
Output added for c_var_weekend
Output added for s_we_min
Output added for s_we_max
Output added for c_we_morning
Output added for c_we_noon
Output added for c_we_afternoon
Output added for c_we_evening
Output added for c_we_night
Output added for c_week_no_min
Output added for s_max_no_min
Output added for c_evening_no_min
Output added for c_morning_no_min
Output added for c_noon_no_min
Output added for c_night_no_min
Output added for r_mean_max
Output added for r_min_mean
Output added for r_night_mean
Output added for r_morning_noo

year,week,average_cons,max_cons,min_cons,average_cons_morning,average_cons_noon,average_cons_afternoon,average_cons_evening,average_cons_night,average_cons_wd,var_cons_wd,min_cons_wd,max_cons_wd,average_cons_wd_morning,average_cons_wd_noon,average_cons_wd_afternoon,average_cons_wd_evening,average_cons_wd_night,average_cons_weekend,var_cons_weekend,min_cons_weekend,max_cons_weekend,average_cons_we_morning,average_cons_we_noon,average_cons_we_afternoon,average_cons_we_evening,average_cons_we_night,cons_week_no_min,cons_max_no_min,cons_evening_no_min,cons_morning_no_min,cons_noon_no_min,cons_night_no_min,ratio_mean_max,ratio_min_mean,ratio_night_mean,…,s_bg_variety,weekday_diff,cons_variance,cons_varianc_wd,cons_variance_we,total_abs_diff,mean_cor,num_peaks,lower_quartile,median,upper_quartile,weekly_avg_max,weekly_avg_min,zero_count,mean_cor_wd,mean_cor_we,correlation,num_small_peaks,num_big_peaks,temp_cons_cor,temp_cons_cor_night,temp_cons_cor_daytime,temp_cons_cor_evening,min_temp_cons_correlation,max_temp_cons_correlation,maxmin_temp_cons_correlation,weekday_weekend_ratio,first_exceeding_1kW_hour,first_exceeding_2kW_hour,first_time_above_mean,time_at_max,time_at_min,mean_residual_stl,mean_autocorrelation,mean_autocorrelation_wd,t_wide_peaks,t_width_peaks
i32,u32,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,i64,f64,f64,f64,f64,f64,u32,f64,f64,f64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,list[f64],list[f64],list[f64],list[i64],list[f64]
2023,48,1.218186,2.296611,0.037381,0.997055,1.352355,1.088006,1.168859,1.357687,null,null,null,null,null,null,null,null,null,1.206937,0.448865,0.037381,2.295452,1.100164,1.264095,1.024271,1.18827,1.379646,1.180804,2.25923,1.131478,0.959674,1.314973,1.320306,0.530427,0.030686,1.114516,…,0.69481,null,0.457443,null,0.457443,200.595622,0.005557,43,0.681094,1.257797,1.789489,2.284654,0.04687,0,0.005557,null,null,49,22,0.081495,0.018789,null,0.023568,0.094506,0.276523,-0.043886,null,null,null,0.0,null,null,[0.235482],[0.010225],null,[134],[3.435897]
2023,22,1.13397,2.297348,0.0108,1.131937,1.057382,1.115154,1.145092,1.17495,1.035243,0.419651,0.043217,2.27326,1.198462,0.917075,0.940635,0.879171,1.736377,1.185074,0.44921,0.0108,2.297348,1.196882,1.188384,1.196934,1.205845,1.176351,1.123171,2.286549,1.134292,1.121138,1.046583,1.16415,0.4936,0.009524,1.036138,…,0.878348,0.809963,0.437807,0.419651,0.440815,284.520963,-0.033497,46,0.562179,1.144,1.668085,2.271526,0.027686,0,-0.033497,null,-0.013125,54,19,-0.040286,-0.035442,0.121724,-0.015117,-0.041024,0.103803,-0.225692,-0.430633,5.0,5.0,1.25,13.0,8.0,null,null,null,null,null
2023,24,1.201697,2.295386,0.001528,1.298298,1.184336,1.348476,1.001102,1.18316,1.201697,0.499721,0.001528,2.295386,1.298298,1.184336,1.348476,1.001102,1.18316,null,null,null,null,null,null,null,null,null,1.20017,2.293858,0.999574,1.29677,1.182808,1.181633,0.523527,0.001271,0.984574,…,0.873448,0.820217,0.499721,0.499721,null,236.222475,0.065482,49,0.618031,1.262347,1.82955,2.116471,0.410353,0,null,0.132669,null,57,22,0.016093,0.011718,-0.052581,-0.005405,2.659989,0.075287,-1.122311,null,0.0,0.666667,0.0,11.5,10.25,[0.224341],[-0.010218],[-0.010218],[126],[3.073171]
2023,50,1.189833,2.293207,0.002078,1.234593,1.199318,1.113629,1.237586,1.17366,1.190817,0.431087,0.002078,2.293207,1.234593,1.199318,1.113629,1.237586,1.17366,0.811984,null,0.811984,0.811984,null,null,null,null,null,1.187756,2.291129,1.235509,1.232515,1.197241,1.171583,0.518851,0.001746,0.986408,…,0.892054,0.788727,0.430337,0.431087,null,302.985785,-0.00687,47,0.654796,1.168502,1.77699,1.986457,0.177051,0,null,-0.095035,null,53,21,-0.093347,-0.033404,-0.013865,0.030683,0.471732,0.17396,-0.825258,0.099519,0.0,0.5,0.0,13.75,11.75,[0.270802],[-0.000739],[-0.00033],[160],[3.018868]
2023,49,1.152842,2.295858,0.00017,1.110777,1.157423,1.22149,1.210373,1.145038,1.123026,0.458061,0.000845,2.

In [9]:
merged_df.columns

['year',
 'week',
 'average_cons',
 'max_cons',
 'min_cons',
 'average_cons_morning',
 'average_cons_noon',
 'average_cons_afternoon',
 'average_cons_evening',
 'average_cons_night',
 'average_cons_wd',
 'var_cons_wd',
 'min_cons_wd',
 'max_cons_wd',
 'average_cons_wd_morning',
 'average_cons_wd_noon',
 'average_cons_wd_afternoon',
 'average_cons_wd_evening',
 'average_cons_wd_night',
 'average_cons_weekend',
 'var_cons_weekend',
 'min_cons_weekend',
 'max_cons_weekend',
 'average_cons_we_morning',
 'average_cons_we_noon',
 'average_cons_we_afternoon',
 'average_cons_we_evening',
 'average_cons_we_night',
 'cons_week_no_min',
 'cons_max_no_min',
 'cons_evening_no_min',
 'cons_morning_no_min',
 'cons_noon_no_min',
 'cons_night_no_min',
 'ratio_mean_max',
 'ratio_min_mean',
 'ratio_night_mean',
 'ratio_morning_noon',
 'ratio_evening_noon',
 'ratio_mean_max_no_min',
 'ratio_evening_noon_no_min',
 'ratio_morning_noon_no_min',
 'ratio_day_night_no_min',
 'ratio_var_wd_we',
 'ratio_min_wd_we